Importation des packages necessaires

In [27]:
from pathlib import Path
from collections import Counter, defaultdict
import json
import time
import pandas as pd

In [28]:
# Dossiers
output_dir = Path("output")
output_dir.mkdir(exist_ok=True)

In [29]:
from e_commerce_franck_sdk import E_commerceClient, E_commerceConfig


config = E_commerceConfig(e_commerce_base_url="https://projet-backend-e-commerce-1.onrender.com")
client = E_commerceClient(config=config)

Initialisation de e_commerceConfig : URL de base = https://projet-backend-e-commerce-1.onrender.com


In [30]:
# Santé de l'API
client.health_check()

{'message': 'API E_commerce opérationnelle'}

In [31]:
# Requête pour récupérer les données d'analyse
analytics=client.get_analytics()
analytics

Analytics(ventes_count=130, clients_count=1000, produits_count=130, commandes_count=1000)

In [32]:
client.get_client(9)

ClientDimBase(id_client=9, nom='Doxsey', prenom='Sacha', gender='Female', email='sdoxsey8@topsy.com', telephone='151-749-2308', adresse='80 SUGAR CREEK CENTER BLVD.')

In [33]:
client.list_clients(5, output_format="pandas")

,id_client,nom,prenom,gender,email,telephone,adresse
0,6,Allery,Celestia,Female,callery5@hugedomains.com,125-956-5543,245 COMMERCIAL STREET
1,7,Daudray,Laraine,Female,ldaudray6@timesonline.co.uk,496-578-9242,15 SAMMY MCGHEE BLVD
2,8,Vasyutochkin,Merilyn,Female,mvasyutochkin7@instagram.com,410-138-4409,200 WEST CONGRESS STREET
3,9,Doxsey,Sacha,Female,sdoxsey8@topsy.com,151-749-2308,80 SUGAR CREEK CENTER BLVD.
4,10,Callacher,Susanna,Female,scallacher9@newsvine.com,949-482-6904,4140 EAST STATE STREET
...,...,...,...,...,...,...,...
95,100,Sommerland,Aksel,Male,asommerland2r@ezinearticles.com,251-876-8204,NY-31-17-0119
96,101,Gavagan,Madella,Female,mgavagan2s@constantcontact.com,100-653-8426,PIAZZA FILIPPO MEDA 4
97,102,Coade,Dwain,Male,dcoade2t@csmonitor.com,283-421-1991,Herzog-Ernst-Ring 49
98,103,Letrange,Amby,Male,aletrange2u@adobe.com,731-138-6487,275 SOUTHWEST THIRD STREET


AVEC SYSTEME DE MISE EN CACHE

In [34]:
analytics

Analytics(ventes_count=130, clients_count=1000, produits_count=130, commandes_count=1000)

Données de Ventes

In [55]:
####### Avec Système de mise en cache #########

# Récupération du nombre total de ventes depuis l'API
api_ventes_count = analytics.ventes_count
print(f"Nombre total de ventes dans l'API : {api_ventes_count}")

ventes_data_file = output_dir / "ventes_df.csv"
meta_file = output_dir / "meta_ventes.json"

# Lecture du fichier méta s'il existe
if meta_file.exists():
    with open(meta_file, "r") as f:
        meta = json.load(f)
    cache_ventes_count = meta.get("ventes_count", 0)
else:
    meta = {}
    cache_ventes_count = 0

# Décision : utiliser le cache ou recalculer
if ventes_data_file.exists() and cache_ventes_count == api_ventes_count:
    print("Chargement des données depuis le cache...")
    ventes_df = pd.read_csv(ventes_data_file)
else:
    print("Mise à jour des données depuis l'API...")

    # MODIFICATION PRINCIPALE : récupération directe en DataFrame
    ventes_df = client.list_ventes(limit=api_ventes_count, output_format="pandas")

    # Sauvegarde des données et du fichier méta
    ventes_df.to_csv(ventes_data_file, index=False)
    with open(meta_file, "w") as f:
        json.dump({"ventes_count": api_ventes_count}, f)

    print("Fichier CSV sauvegardé avec succès.")

# Aperçu
print(ventes_df.head())


Nombre total de ventes dans l'API : 130
Mise à jour des données depuis l'API...
Fichier CSV sauvegardé avec succès.
   id_commande  id_client  id_produit  quantite_totale  vente_totale
0            9          9           9              655       2833.47
1           19         19          19              473       1265.40
2           35         35          35              924       8759.21
3           45         45          45              501       2122.33
4           52         52          52                7       8551.37


In [48]:
ventes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_commande      130 non-null    int64  
 1   id_client        130 non-null    int64  
 2   id_produit       130 non-null    int64  
 3   quantite_totale  130 non-null    int64  
 4   vente_totale     130 non-null    float64
dtypes: float64(1), int64(4)
memory usage: 5.2 KB


In [49]:
ventes_df.isnull().sum()

id_commande        0
id_client          0
id_produit         0
quantite_totale    0
vente_totale       0
dtype: int64

Données de Clients

In [50]:
analytics.clients_count

1000

In [ ]:
####### Avec Système de mise en cache #########

# Récupération du nombre total de clients depuis l'API
api_clients_count = analytics.clients_count
print(f"Nombre total de clients dans l'API : {api_clients_count}")

clients_data_file = output_dir / "clients_df.csv"
meta_file = output_dir / "meta_clients.json"

# Lecture du fichier méta s'il existe
if meta_file.exists():
    with open(meta_file, "r") as f:
        meta = json.load(f)
    cache_clients_count = meta.get("clients_count", 0)
else:
    meta = {}
    cache_clients_count = 0

# Décision : utiliser le cache ou recalculer
if clients_data_file.exists() and cache_clients_count == api_clients_count:
    print("Chargement des données depuis le cache...")
    clients_df = pd.read_csv(clients_data_file)
else:
    print("Mise à jour des données depuis l'API...")

    # Paramètres pour le batching
    limit = 100
    skip = 0
    clients_rows = []

    while True:
        try:
            batch = client.list_clients(skip=skip, limit=limit, output_format="dict")
        except Exception as e:
            print(f"Erreur à skip={skip} : {e}. Nouvelle tentative dans 5 secondes...")
            time.sleep(5)
            continue

        if not batch:
            break

        clients_rows.extend(batch)
        skip += limit
        time.sleep(0.5)  # Pour respecter l’API

    # Création du DataFrame
    clients_df = pd.DataFrame(clients_rows)

    # Sauvegarde des données et du fichier méta
    clients_df.to_csv(clients_data_file, index=False)
    with open(meta_file, "w") as f:
        json.dump({"clients_count": api_clients_count}, f)

    print("Fichier CSV sauvegardé avec succès.")



In [57]:
# Aperçu
clients_df.head()

,id_client,nom,prenom,gender,email,telephone,adresse
0,1,Juan,Monti,Male,mjuan0@cnn.com,365-532-8326,Rosenstraße 2
1,2,Blampey,Bobby,Male,bblampey1@bigcartel.com,428-681-3229,"Market Street, Dundalk,"
2,3,Bleue,Glory,Female,gbleue2@weibo.com,235-182-7891,"260, Okolovrasten Pat Blvd."
3,4,Van der Baaren,Rooney,Male,rvanderbaaren3@salon.com,537-745-0007,4140 EAST STATE STREET
4,5,Wynne,Ninette,Female,nwynne4@samsung.com,872-153-3637,Burgstraße 1


Données Produits

In [60]:
client.list_produits(output_format="pandas").head()

,id_produit,nom,categorie
0,9,Electric Wax Warmer,Maison
1,19,Hand Crank Blender,Cuisine
2,35,Non-Stick Grill Pan,Cuisine
3,45,Scented Soy Candles,Maison
4,52,Luxury Memory Foam Mattress Topper,Maison


In [61]:
client.list_produits(output_format="pandas").shape

(100, 3)

In [63]:
####### Avec Système de mise en cache #########

# Récupération du nombre total de produits depuis l'API 
api_produits_count = analytics.produits_count
print(f"Nombre total de produits dans l'API : {api_produits_count}")

produits_data_file = output_dir / "produits_df.csv"
meta_file = output_dir / "meta_produits.json"

# Lecture du fichier méta s'il existe
if meta_file.exists():
    with open(meta_file, "r") as f:
        meta = json.load(f)
    cache_produits_count = meta.get("produits_count", 0)
else:
    meta = {}
    cache_produits_count = 0

# Décision : utiliser le cache ou recalculer
if produits_data_file.exists() and cache_produits_count == api_produits_count:
    print("Chargement des données depuis le cache...")
    produits_df = pd.read_csv(produits_data_file)
else:
    print("Mise à jour des données depuis l'API...")

    # Paramètres pour le batching
    limit = 50
    skip = 0
    produits_rows = []

    while True:
        try:
            batch = client.list_produits(skip=skip, limit=limit, output_format="dict")
        except Exception as e:
            print(f"Erreur à skip={skip} : {e}. Nouvelle tentative dans 5 secondes...")
            time.sleep(5)
            continue

        if not batch:
            break

        produits_rows.extend(batch)
        skip += limit
        time.sleep(0.5)  # Pour respecter l’API

    # Création du DataFrame
    produits_df = pd.DataFrame(produits_rows)

    # Sauvegarde des données et du fichier méta
    produits_df.to_csv(produits_data_file, index=False)
    with open(meta_file, "w") as f:
        json.dump({"produits_count": api_produits_count}, f)

    print("Fichier CSV sauvegardé avec succès.")



Nombre total de produits dans l'API : 130
Mise à jour des données depuis l'API...
Fichier CSV sauvegardé avec succès.


Données Commandes

In [64]:
analytics.commandes_count

1000

In [65]:
client.list_commandes(output_format="pandas").head()

,id_commande,date_commande,statut
0,1,2021-07-31 00:00:00,annulee
1,2,2021-06-24 00:00:00,annulee
2,3,2021-12-01 00:00:00,annulee
3,4,2020-12-28 00:00:00,expediee
4,5,2022-04-18 00:00:00,en attente


In [66]:
####### Avec Système de mise en cache #########

# Récupération du nombre total de commandes depuis l'API 
api_commandes_count = analytics.commandes_count
print(f"Nombre total de commandes dans l'API : {api_commandes_count}")

commandes_data_file = output_dir / "commandes_df.csv"
meta_file = output_dir / "meta_commandes.json"

# Lecture du fichier méta s'il existe
if meta_file.exists():
    with open(meta_file, "r") as f:
        meta = json.load(f)
    cache_commandes_count = meta.get("commandes_count", 0)
else:
    meta = {}
    cache_commandes_count = 0

# Décision : utiliser le cache ou recalculer
if commandes_data_file.exists() and cache_commandes_count == api_commandes_count:
    print("Chargement des données depuis le cache...")
    commandes_df = pd.read_csv(commandes_data_file)
else:
    print("Mise à jour des données depuis l'API...")

    # Paramètres pour le batching
    limit = 50
    skip = 0
    commandes_rows = []

    while True:
        try:
            batch = client.list_commandes(skip=skip, limit=limit, output_format="dict")
        except Exception as e:
            print(f"Erreur à skip={skip} : {e}. Nouvelle tentative dans 5 secondes...")
            time.sleep(5)
            continue

        if not batch:
            break

        commandes_rows.extend(batch)
        skip += limit
        time.sleep(0.5)  # Pour respecter l’API

    # Création du DataFrame
    commandes_df = pd.DataFrame(commandes_rows)

    # Sauvegarde des données et du fichier méta
    commandes_df.to_csv(commandes_data_file, index=False)
    with open(meta_file, "w") as f:
        json.dump({"commandes_count": api_commandes_count}, f)

    print("Fichier CSV sauvegardé avec succès.")



Nombre total de commandes dans l'API : 1000
Mise à jour des données depuis l'API...
Fichier CSV sauvegardé avec succès.


In [67]:
commandes_df.head()

,id_commande,date_commande,statut
0,1,2021-07-31 00:00:00,annulee
1,2,2021-06-24 00:00:00,annulee
2,3,2021-12-01 00:00:00,annulee
3,4,2020-12-28 00:00:00,expediee
4,5,2022-04-18 00:00:00,en attente


In [68]:
commandes_df.isnull().sum()

id_commande      0
date_commande    0
statut           0
dtype: int64

In [69]:
commandes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id_commande    1000 non-null   int64 
 1   date_commande  1000 non-null   object
 2   statut         1000 non-null   object
dtypes: int64(1), object(2)
memory usage: 23.6+ KB


In [70]:
commandes_df.duplicated().sum()

np.int64(0)